In [1]:
#Importing pyomo and other needed packages

import pyomo.environ as pyo
import pyomo.opt as po
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from pyomo.util.infeasible import log_infeasible_constraints

# Importing Data from Excel and Transforming to Dictionaries

In [2]:
#######################################################
###############Inputting Parameter Data ###############
#######################################################

#All data is inputted from excel transformed to pandas dataframe and to dictionaries with .to_dict() command

#Network Generration Parameters (considered unlimited so set to 1,000,000), however model allows to put the limit

Network_Power=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=0)

NPD_df=pd.DataFrame(Network_Power)


PN_min=NPD_df.set_index('B')['PN_min'].to_dict()
PN_max=NPD_df.set_index('B')['PN_max'].to_dict()

#Parameters for Line Capacity limits 

Line_Par=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=1)

LF_df=pd.DataFrame(Line_Par)

fmin_l=LF_df.set_index('L')['fmin_l'].to_dict()
fmax_l=LF_df.set_index('L')['fmax_l'].to_dict()

#Reactance Matrix

Reactance=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=2)

XL_df=pd.DataFrame(Reactance)

XL=XL_df.set_index('L')['X'].to_dict()

#A matrix line-adjacency matrix

Line_adjacency=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=3)

ALIM_df=pd.DataFrame(Line_adjacency)

A_matrixIM=ALIM_df.set_index('L', inplace=False)

AIM=A_matrixIM.stack().to_dict()

Line_adjacencyOM=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=4)

ALOM_df=pd.DataFrame(Line_adjacencyOM)

A_matrixOM=ALOM_df.set_index('L', inplace=False)

AOM=A_matrixOM.stack().to_dict()

Line_adjacencyB=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=5)

ALB_df=pd.DataFrame(Line_adjacencyB)

A_matrixB=ALB_df.set_index('L', inplace=False)

AB=A_matrixB.stack().to_dict()

#Islanded Microgrid Generation

IM_ParG=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=6)

IM_G=pd.DataFrame(IM_ParG)

PImin_tmg=IM_G.set_index(['T','M', 'G'])['Pimin_tmg'].to_dict()

PImax_tmg=IM_G.set_index(['T','M', 'G'])['Pimax_tmg'].to_dict()

RUI_tmg=IM_G.set_index(['T','M', 'G'])['RUI_tmg'].to_dict()

RDI_tmg=IM_G.set_index(['T','M', 'G'])['RDI_tmg'].to_dict()

#Islanded Microgrid Storage Parameters

IM_ParS=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=7)

IM_S=pd.DataFrame(IM_ParS)

Emin_m=IM_S.set_index(['M'])['Eimin_m'].to_dict()

Emax_m=IM_S.set_index(['M'])['Eimax_m'].to_dict()

PISDmax_m=IM_S.set_index(['M'])['PISDmax_m'].to_dict()

PISCmax_m=IM_S.set_index(['M'])['PISCmax_m'].to_dict()

sigmaIS_m=IM_S.set_index(['M'])['sigmaIS_m'].to_dict()

#Outaged Microgrid Generation 

OM_ParG=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=8)

OM_G=pd.DataFrame(OM_ParG)

POmin_tng=OM_G.set_index(['T','N', 'G'])['Pomin_tng'].to_dict()

POmax_tng=OM_G.set_index(['T','N', 'G'])['Pomax_tng'].to_dict()

RUO_tng=OM_G.set_index(['T','N', 'G'])['RUO_tng'].to_dict()

RDO_tng=OM_G.set_index(['T','N', 'G'])['RDO_tng'].to_dict()

#Outaged Storage Parameters 

OM_ParS=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=9)

OM_S=pd.DataFrame(OM_ParS)

Emin_n=OM_S.set_index(['N'])['EOmin'].to_dict()

Emax_n=OM_S.set_index(['N'])['EOmax'].to_dict()

POSDmax_n=OM_S.set_index(['N'])['POSD_max'].to_dict()

POSCmax_n=OM_S.set_index(['N'])['POSC_max'].to_dict()

sigmaOS_n=OM_S.set_index(['N'])['sigmaOS_n'].to_dict()

#Islanded Microgrid Load 

DIM_m=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=10)

DIM_df=pd.DataFrame(DIM_m)

DIM_matrix=DIM_df.set_index('T', inplace=False)

DIM=DIM_matrix.stack().to_dict()

#Outaged Microgrid Load

DOM_n=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=11)

DOM_df=pd.DataFrame(DOM_n)

DOM=DOM_df.set_index('N')['Load'].to_dict()

SUOM_dat=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=12)

SUOM_df=pd.DataFrame(SUOM_dat)

SUOM=SUOM_df.set_index('OM')['SUOM'].to_dict()

#Outaged Microgrid Delta Load (for CLPU)

DeltaO_tk=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=13)

DeltaO_df=pd.DataFrame(DeltaO_tk)

DeltaO_matrix=DeltaO_df.set_index('T')

DeltaO=DeltaO_matrix.stack().to_dict()

#Bus Load

DBL_b=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=14)

DB_df=pd.DataFrame(DBL_b)

DB=DB_df.set_index('B')['Load'].to_dict()

SUB_dat=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=15)

SUB_df=pd.DataFrame(SUB_dat)

SUB=SUB_df.set_index('B')['SUB'].to_dict()

#Bus Delta Load (for CLPU)

DeltaB_tk=pd.read_excel('Model Parameters_V1.xlsx', sheet_name=16)

DeltaB_df=pd.DataFrame(DeltaB_tk)

DeltaB_matrix=DeltaB_df.set_index('T')

DeltaB=DeltaB_matrix.stack().to_dict()

# Declaration: Model, Sets, Parameters, Variables

In [3]:
############ Defining a concrete model ###########

model = pyo.ConcreteModel()


########## Declaration of Sets ############

model.T=pyo.RangeSet(1,12) #Set of time periods
model.M=pyo.RangeSet(1,3) #Set of microgrids in islanded mode at t=0 (CLPU after connection) 
model.N=pyo.RangeSet(1,3) #Set of microgrids in not energized at t=0 (energized with diversified load)
model.B=pyo.RangeSet(1,3) #Set of buses
model.L=pyo.RangeSet(1,8) #Set of lines connecting microgrids and buses to main grid
model.G=pyo.RangeSet(1,2) #Set of generators 
model.K=pyo.RangeSet(1,12) #set of samples for CLPU estimation

########Declaration of Parameters#########

#Power generation limit parameters 

model.Pmin_tmg=pyo.Param(model.T, model.M, model.G, initialize=PImin_tmg) #Lower bound for power generation for power plant G in islanded MG M
model.Pmax_tmg=pyo.Param(model.T, model.M, model.G, initialize=PImax_tmg) #Upper bound for power generation for power plant G in islanded MG M
model.Pmin_tng=pyo.Param(model.T, model.N, model.G, initialize=POmin_tng) #Lower bound for power generation for power plant G in outaged MG N
model.Pmax_tng=pyo.Param(model.T, model.N, model.G, initialize=POmax_tng) #Upper bound for power generation for power plant G in outaged MG N
model.PNmin=pyo.Param(model.B, initialize=PN_min) #lower bound on power generation in main grid
model.PNmax=pyo.Param(model.B, initialize=PN_max) #Upper bound on power generation in main grid

#Power generation ramping-up and ramping-down parameters

model.RU_tmg=pyo.Param(model.T, model.M, model.G, initialize=RUI_tmg) #Ramping up limit for power generator G in islanded MG M
model.RD_tmg=pyo.Param(model.T, model.M, model.G, initialize=RDI_tmg) #Ramping down limit for power generator G in islanded MG M
model.RU_tng=pyo.Param(model.T, model.N, model.G, initialize=RUO_tng) #Ramping up limit for power generator G in outaged MG N
model.RD_tng=pyo.Param(model.T, model.N, model.G, initialize=RDO_tng) #Ramping down limit for power generator G in outaged MG N

#Power Storage parameters

model.Emin_m=pyo.Param(model.M, initialize=Emin_m) #Minimum energy sotrage capacity parameter for IM M
model.Emax_m=pyo.Param(model.M, initialize=Emax_m) #Maximum energy sotrage capacity parameter for IM M
model.Emin_n=pyo.Param(model.N, initialize=Emin_n) #Minimum energy sotrage capacity parameter for OM N
model.Emax_n=pyo.Param(model.N, initialize=Emax_n) #Maximum energy sotrage capacity parameter for OM N
model.PSCmax_m=pyo.Param(model.M, initialize=PISCmax_m) #Charging per period for IM M
model.PSDmax_m=pyo.Param(model.M, initialize=PISDmax_m) #Charging per period for IM M
model.PSCmax_n=pyo.Param(model.N, initialize=POSCmax_n) #Charging per period for IM N
model.PSDmax_n=pyo.Param(model.N, initialize=POSDmax_n) #Charging per period for IM N
model.SigmaS_m=pyo.Param(model.M, initialize=sigmaIS_m) # Storage charge/discharge efficiency
model.SigmaS_n=pyo.Param(model.N, initialize=sigmaOS_n) # Storage charge/discharge efficiency

#Power Flow Parameters
model.fmax_l=pyo.Param(model.L, initialize=fmax_l) #maximum powerflow for power line L
model.fmin_l=pyo.Param(model.L, initialize=fmin_l) #minimum powerflow for power line L
model.X_l=pyo.Param(model.L, initialize=XL) #reactance matrix for power line L
model.AM_lm=pyo.Param(model.L, model.M, initialize=AIM) #Line adjacency coefficient for M (+1 for OM and B, -1 for IM)
model.AN_ln=pyo.Param(model.L, model.N, initialize=AOM) #Line adjacency coefficient for N (+1 for OM and B, -1 for IM)
model.AB_lb=pyo.Param(model.L, model.B, initialize=AB) #Line adjacency coefficient for N (+1 for OM and B, -1 for IM)

#Load Parameters 
model.DIM_tm=pyo.Param(model.T, model.M, initialize=DIM) #diversified load at islanded microgrid
model.DOML_n=pyo.Param(model.N, initialize=DOM) #diversified load for outaged microgrid
model.DBL_b=pyo.Param(model.B, initialize=DB) #base load for for bus b
model.SUO_n=pyo.Param(model.N, initialize=SUOM) #Undifersified (peak) load for outaged microgrid n
model.SUB_b=pyo.Param(model.B, initialize=SUB) #Undifersified (peak) base for bus b
model.deltaDO_kn=pyo.Param(model.K, model.N, initialize=DeltaO) #change in load after CLPU for network n
model.deltaDB_kb=pyo.Param(model.K, model.B, initialize=DeltaB) #changein load after CLPU for bus b

##########Declaration of Variables###########

#Power generation variables

model.PIMG_tmg=pyo.Var(model.T, model.M, model.G, domain=pyo.PositiveReals) #Power generation level at time t, for generator g at islanded microgrid M
model.POMG_tng=pyo.Var(model.T, model.N, model.G, domain=pyo.PositiveReals)  #Power generation level at time t, for generator g at islanded microgrid N
model.PN_tb=pyo.Var(model.T, model.B, domain=pyo.PositiveReals) #Power generation level at time t from main Grid

#Power distribution by line 

model.PL_tl=pyo.Var(model.T, model.L, domain=pyo.PositiveReals) #Power distribution at time t by line l
model.deltaIM_tm=pyo.Var(model.T, model.M, domain=pyo.Reals) #bus angle for islanded microgrid at time t 
model.deltaOM_tn=pyo.Var(model.T, model.N, domain=pyo.Reals) #bus angle for islanded microgrid at time t
model.deltaB_tb=pyo.Var(model.T, model.B, domain=pyo.Reals) #bus angle for buse b at time t

#Storage Variables

model.EIM_tm=pyo.Var(model.T, model.M, domain=pyo.PositiveReals) #Storage level at time t for islanded microgrid M
model.EOM_tn=pyo.Var(model.T, model.N, domain=pyo.PositiveReals) #Storage level at time t for outaged microgrid N
model.PSD_tm=pyo.Var(model.T, model.M, domain=pyo.PositiveReals) #Storage discharge at time t for islanded microgrid M
model.PSD_tn=pyo.Var(model.T, model.N, domain=pyo.PositiveReals) #Storage discharge at time t for outaged microgrid N
model.PSC_tm=pyo.Var(model.T, model.M, domain=pyo.PositiveReals) #Storage charge at time t for islanded microgrid M
model.PSC_tn=pyo.Var(model.T, model.N, domain=pyo.PositiveReals) #Storage charge at time t for outaged microgrid N
model.sIM_tm=pyo.Var(model.T, model.M, within=pyo.Binary) #Srorage charge status at time t for islanded microgrid M
model.sOM_tn=pyo.Var(model.T, model.N, within=pyo.Binary) #Storage charge status at time t for outaged microgrid N
model.dIM_tm=pyo.Var(model.T, model.M, within=pyo.Binary) #Storage discharge status at time t for islanded microgrid M
model.dOM_tn=pyo.Var(model.T, model.N, within=pyo.Binary) #Storage discharge status at time t for islanded microgrid  N

#Load Variables

model.DB_tb=pyo.Var(model.T, model.B, domain=pyo.PositiveReals) #load level at bus b
model.DOM_tn=pyo.Var(model.T, model.N, domain=pyo.PositiveReals) #load level at outaged microgrid N


#Grid connection and energizing

model.z_tn=pyo.Var(model.T, model.N, within=pyo.Binary) #binary variable 1 if outaged microgrid N is connected to main grid
model.u_tb=pyo.Var(model.T, model.B, within=pyo.Binary) #binary variable 1 if bus b is energized

# Model: Objective and Constraints

In [4]:
######################################################################
######################    Objective Function (3.1)   #################
######################################################################

obj_expr=(sum(sum(model.z_tn[t, n] for n in model.N)+
              sum(model.u_tb[t,b] for b in model.B) for t in model.T))

model.obj=pyo.Objective(sense=pyo.maximize, expr=obj_expr)

###############################################################
######################    Constraints    ######################
###############################################################



#####################  KCL Constraints (3.5) #########################

model.KCL=pyo.ConstraintList()

for t in model.T:
    for b in model.B:
        lhs=model.PN_tb[t,b]+sum(model.AB_lb[l,b]*model.PL_tl[t,l] for l in model.L)
        rhs=model.DB_tb[t,b]
        model.KCL.add(lhs==rhs)



for t in model.T:
    for n in model.N:
        lhs=(sum(model.POMG_tng[t,n,g] for g in model.G)
             +(model.PSD_tn[t,n]-model.PSC_tn[t,n])+sum(model.AN_ln[l,n]*model.PL_tl[t,l] for l in model.L))
        rhs=model.DOM_tn[t,n]
        model.KCL.add(lhs==rhs)


for t in model.T:
    for m in model.M:
        lhs=(sum(model.PIMG_tmg[t,m,g] for g in model.G)
             +(model.PSD_tm[t,m]-model.PSC_tm[t,m])+sum(model.AM_lm[l,m]*model.PL_tl[t,l] for l in model.L))
        rhs=model.DIM_tm[t,m]
        model.KCL.add(lhs==rhs)
        
#################### KVL Constraint (3.7) #########################

model.KVL=pyo.ConstraintList()

## KVL for IM ##


for t in model.T:
    for l in model.L:
        lhs=(model.X_l[l]*model.PL_tl[t,l]+
             sum(model.AM_lm[l,m]*model.deltaIM_tm[t,m] for m in model.M)+
             sum(model.AN_ln[l,n]*model.deltaOM_tn[t,n] for n in model.N)+
             sum(model.AB_lb[l,b]*model.deltaB_tb[t,b] for b in model.B))
        rhs=0
        model.KVL.add(lhs==rhs)

################### Power Distribution Constraint (3.6) ###################

model.PDC=pyo.ConstraintList()

for t in model.T:
    for l in model.L:
        model.PDC.add(model.fmin_l[l]<=model.PL_tl[t,l])
        model.PDC.add(model.fmax_l[l]>=model.PL_tl[t,l])
        
#################  Power Genration constraints (3.8) #################### 

model.PGC=pyo.ConstraintList()

for t in model.T:
    for b in model.B:
        model.PGC.add(model.PNmin[b]<=model.PN_tb[t,b])
        model.PGC.add(model.PNmax[b]>=model.PN_tb[t,b])
    
for t in model.T:
    for m in model.M:
        for g in model.G:
            model.PGC.add(model.Pmin_tmg[t,m,g]<=model.PIMG_tmg[t,m,g])
            model.PGC.add(model.Pmax_tmg[t,m,g]>=model.PIMG_tmg[t,m,g])
for t in model.T:
    for n in model.N:
        for g in model.G:
            model.PGC.add(model.Pmin_tng[t,n,g]<=model.POMG_tng[t,n,g])
            model.PGC.add(model.Pmax_tng[t,n,g]>=model.POMG_tng[t,n,g])


############## Ramping up/Down Constraint for microgrids (3.9) ############## 

#Function for Ramping Up constraint defining generation to be equal demand at period t=0

def RUCIM(model,t,m,g):
    a=model.PIMG_tmg[t,m,g]
    if t==1:
        b=model.DIM_tm[1,m]
    else:
        b=model.PIMG_tmg[t-1,m,g]
    return a-b<=model.RU_tmg[t,m,g]

#Function for Ramping Down constraint defining generation to be equal demand at period t=0

def RDCIM(model,t,m,g):
    a=model.PIMG_tmg[t,m,g]
    if t==1:
        b=model.DIM_tm[1,m]
    else:
        b=model.PIMG_tmg[t-1,m,g]
    return b-a<=model.RD_tmg[t,m,g]

model.RUCIM=pyo.Constraint(model.T, model.M, model.G, rule=RUCIM)
model.RDCIM=pyo.Constraint(model.T, model.M, model.G, rule=RDCIM)

#Function for Ramping up constraint for outaged microgrid defining generation to be 0 at t=0


def RUCOM(model, t, n, g):
    a=model.POMG_tng[t,n,g]
    if t==1:
        b=0
    else:
        b=model.POMG_tng[t-1,n,g]
    return a-b<=model.RU_tng[t,n,g]

#Function for Ramping down constraint for outaged microgrid defining generation to be 0 at t=0

def RDCOM(model, t, n, g):
    a=model.POMG_tng[t,n,g]
    if t==1:
        b=0
    else:
        b=model.POMG_tng[t-1,n,g]
    return b-a<=model.RD_tng[t,n,g]

model.RUCOM=pyo.Constraint(model.T, model.N, model.G, rule=RUCOM)
model.RDCOM=pyo.Constraint(model.T, model.N, model.G, rule=RDCOM)

############## Storage Constraints (3.10) ############## 

#Islanded microgrid storage constraints

model.IMSC=pyo.ConstraintList()

for t in model.T:
    for m in model.M:
        lhs1=model.PSC_tm[t,m]
        rhs1=model.PSCmax_m[m]*model.sIM_tm[t,m]
        model.IMSC.add(lhs1<=rhs1)
        lhs2=model.PSD_tm[t,m]
        rhs2=model.PSDmax_m[m]*model.dIM_tm[t,m]
        model.IMSC.add(lhs2<=rhs2)
        lhs3=model.Emin_m[m]
        rhs3=model.Emax_m[m]
        model.IMSC.add(lhs3<=model.EIM_tm[t,m])
        model.IMSC.add(rhs3>=model.EIM_tm[t,m])
        lhs4=model.sIM_tm[t,m]+model.dIM_tm[t,m]
        rhs4=1
        model.IMSC.add(lhs4<=rhs4)
        
#Constraint for Storage update

def ESUIM(model,t,m):
    rhs=model.EIM_tm[t,m]+model.SigmaS_m[m]*model.PSC_tm[t,m]-(1/model.SigmaS_m[m])*model.PSD_tm[t,m]
    if t==12:
        lhs=model.EIM_tm[1,m]
    else:
        lhs=model.EIM_tm[t+1,m]
    return lhs==rhs

model.ESUCIM=pyo.Constraint(model.T, model.M, rule=ESUIM)        

#Outaged microgrid storage constraints

model.OMSC=pyo.ConstraintList()

for t in model.T:
    for n in model.N:
        lhs1=model.PSC_tn[t,n]
        rhs1=model.PSCmax_n[n]*model.sOM_tn[t,n]
        model.IMSC.add(lhs1<=rhs1)
        lhs2=model.PSD_tn[t,n]
        rhs2=model.PSDmax_n[n]*model.dOM_tn[t,n]
        model.IMSC.add(lhs2<=rhs2)
        lhs3=model.Emin_n[n]
        rhs3=model.Emax_n[n]
        model.IMSC.add(lhs3<=model.EOM_tn[t,n])
        model.IMSC.add(rhs3>=model.EOM_tn[t,n])
        lhs4=model.sOM_tn[t,n]+model.dOM_tn[t,n]
        rhs4=1
        model.IMSC.add(lhs4<=rhs4)

#Constraint for storage update

def ESUOM(model,t,n):
    rhs=model.EOM_tn[t,n]+model.SigmaS_n[n]*model.PSC_tn[t,n]-(1/model.SigmaS_n[n])*model.PSD_tn[t,n]
    if t==12:
        lhs=model.EOM_tn[1,n]
    else:
        lhs=model.EOM_tn[t+1,n]
    return lhs==rhs

model.ESUCOM=pyo.Constraint(model.T, model.N, rule=ESUOM)

############## Demand Constraints (3.2 - 3.3) ############## 

#Connection fixing for OM (3.2)

def CFOM(model,t,n):
    lhs=model.z_tn[t,n]
    if t==1:
        rhs=0
    else:
        rhs=model.z_tn[t-1,n]
    return lhs>=rhs

model.CFCOM=pyo.Constraint(model.T, model.N, rule=CFOM)

#Connection fixing for Buses (3.2)

def CFB(model,t,b):
    lhs=model.u_tb[t,b]
    if t==1:
        rhs=0
    else:
        rhs=model.u_tb[t-1,b]
    return lhs>=rhs

model.CFCB=pyo.Constraint(model.T, model.B, rule=CFB)

#Demand constraints (3.3)

def DUOM(model,t,n,k):   
    lhs=model.DOM_tn[t,n]
    rhs=(model.DOML_n[n]*(model.SUO_n[n]*model.z_tn[t,n]-
                          sum(model.deltaDO_kn[k,n]*model.z_tn[t-k+1,n] for k in range(1,t)))) 
    return lhs==rhs

model.DUOM=pyo.Constraint(model.T, model.N, model.K, rule=DUOM)


def DUB(model,t,b,k):
    lhs=model.DB_tb[t,b]
    rhs=(model.DBL_b[b]*(model.SUB_b[b]*model.u_tb[t,b]-
                        sum(model.deltaDB_kb[k,b]*model.u_tb[t-k+1,b] for k in range(1,t))))
    return lhs==rhs

model.DUB=pyo.Constraint(model.T, model.B, model.K, rule=DUB)

### Constraints to ensure all buses and OMs are turned at the final period (3.4) ###

model.DminC=pyo.ConstraintList()

for n in model.N:
    lhs=sum(model.DOM_tn[t,n] for t in model.T)
    rhs=model.DOML_n[n]
    model.DminC.add(lhs>=rhs)
for b in model.B:
    lhs=sum(model.DB_tb[t,b] for t in model.T)
    rhs=model.DBL_b[b]
    model.DminC.add(lhs>=rhs)
    
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Solving Model

In [5]:
opt = pyo.SolverFactory('gurobi').solve(model, tee=True ,report_timing=True)



opt.write()

        0.09 seconds required to write file
        0.09 seconds required for presolve

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-09-01
Using license file /Users/levanpavlenishvili/gurobi.lic
Read LP format model from file /var/folders/hk/k1m7cwy5195fxnz2878q0lpr0000gn/T/tmp2nr3jq7o.pyomo.lp
Reading time = 0.01 seconds
x889: 2419 rows, 889 columns, 7699 nonzeros
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2419 rows, 889 columns and 7699 nonzeros
Model fingerprint: 0xe623c162
Variable types: 673 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Pres

# Model Results

In [6]:
####### Model Results to Dataframes #####


#Power Generation

PN_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PN_tb, names=['t', 'b']))
PN_df['GEN']=[pyo.value(model.PN_tb[key]) for key in PN_df.index]

PN_r=PN_df.unstack(level='t')

PIMG_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PIMG_tmg, names=['t','m', 'g']))
PIMG_df['GEN']=[pyo.value(model.PIMG_tmg[key]) for key in PIMG_df.index]

PIMG_r=PIMG_df.unstack(level='t')

POMG_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.POMG_tng, names=['t', 'n', 'g']))
POMG_df['Gen']=[pyo.value(model.POMG_tng[key]) for key in POMG_df.index]

POMG_r=POMG_df.unstack(level='t')

#Power Flow

PL_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PL_tl, names=['t', 'l']))
PL_df['Flow']=[pyo.value(model.PL_tl[key]) for key in PL_df.index]

PL_r=PL_df.unstack(level='t')

deltaIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.deltaIM_tm, names=['t', 'm']))
deltaIM_df['Angle']=[pyo.value(model.deltaIM_tm[key]) for key in deltaIM_df.index]

deltaIM_r=deltaIM_df.unstack(level='t')

deltaOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.deltaOM_tn, names=['t', 'n']))
deltaOM_df['Angle']=[pyo.value(model.deltaOM_tn[key]) for key in deltaOM_df.index]

deltaOM_r=deltaOM_df.unstack(level='t')

deltaB_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.deltaB_tb, names=['t', 'b']))
deltaB_df['Angle']=[pyo.value(model.deltaB_tb[key]) for key in deltaB_df.index]

deltaB_r=deltaB_df.unstack(level='t')

#Storage Variables

EIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.EIM_tm, names=['t', 'm']))
EIM_df['Storage Level']=[pyo.value(model.EIM_tm[key]) for key in EIM_df.index]

EIM_r=EIM_df.unstack(level='t')

EOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.EOM_tn, names=['t', 'n']))
EOM_df['Storage Level']=[pyo.value(model.EOM_tn[key]) for key in EOM_df.index]

EOM_r=EOM_df.unstack(level='t')

PSDIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PSD_tm, names=['t', 'm']))
PSDIM_df['Power Discharge']=[pyo.value(model.PSD_tm[key]) for key in PSDIM_df.index]

PSDIM_r=PSDIM_df.unstack(level='t')

PSDOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PSD_tn, names=['t', 'n']))
PSDOM_df['Power Discharge']=[pyo.value(model.PSD_tn[key]) for key in PSDOM_df.index]

PSDOM_r=PSDOM_df.unstack(level='t')

PSCIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PSC_tm, names=['t', 'm']))
PSCIM_df['Power Charge']=[pyo.value(model.PSC_tm[key]) for key in PSCIM_df.index]

PSCIM_r=PSCIM_df.unstack(level='t')

PSCOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.PSC_tn, names=['t', 'n']))
PSCOM_df['Power Charge']=[pyo.value(model.PSC_tn[key]) for key in PSCOM_df.index]

PSCOM_r=PSCOM_df.unstack(level='t')

sIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.sIM_tm, names=['t', 'm']))
sIM_df['Charge']=[pyo.value(model.sIM_tm[key]) for key in sIM_df.index]

sIM_r=sIM_df.unstack(level='t')

dIM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.dIM_tm, names=['t', 'm']))
dIM_df['Discharge']=[pyo.value(model.dIM_tm[key]) for key in dIM_df.index]

dIM_r=dIM_df.unstack(level='t')

sOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.sOM_tn, names=['t', 'n']))
sOM_df['Charge']=[pyo.value(model.sOM_tn[key]) for key in sOM_df.index]

sOM_r=sOM_df.unstack(level='t')

dOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.dOM_tn, names=['t', 'n']))
dOM_df['Discharge']=[pyo.value(model.dOM_tn[key]) for key in dOM_df.index]

dOM_r=dOM_df.unstack(level='t')

#Load Levels

DB_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.DB_tb, names=['t', 'b']))
DB_df['Load']=[pyo.value(model.DB_tb[key]) for key in DB_df.index]

DB_r=DB_df.unstack(level='t')

DOM_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.DOM_tn, names=['t', 'n']))
DOM_df['Load']=[pyo.value(model.DOM_tn[key]) for key in DOM_df.index]

DOM_r=DOM_df.unstack(level='t')

z_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.z_tn, names=['t', 'n']))
z_df['Connection']=[pyo.value(model.z_tn[key]) for key in z_df.index]

z_r=z_df.unstack(level='t')

u_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(model.u_tb, names=['t', 'b']))
u_df['Connection']=[pyo.value(model.u_tb[key]) for key in u_df.index]

u_r=u_df.unstack(level='t')

In [7]:
##### Putting Output in Excel File #####

with pd.ExcelWriter('M_Output.xlsx') as writer:
    PN_r.to_excel(writer, sheet_name='Sheet1')
    POMG_r.to_excel(writer, sheet_name='Sheet2')
    PIMG_r.to_excel(writer, sheet_name='Sheet3')
    DOM_r.to_excel(writer, sheet_name='Sheet4')
    DB_r.to_excel(writer, sheet_name='Sheet5')
    PL_r.to_excel(writer, sheet_name='Sheet6')
    deltaIM_r.to_excel(writer, sheet_name='Sheet7')
    deltaOM_r.to_excel(writer, sheet_name='Sheet8')
    deltaB_r.to_excel(writer, sheet_name='Sheet9')
    EIM_r.to_excel(writer, sheet_name='Sheet10')
    PSDIM_r.to_excel(writer, sheet_name='Sheet11')
    PSCIM_r.to_excel(writer, sheet_name='Sheet12')
    EOM_r.to_excel(writer, sheet_name='Sheet13')
    PSDOM_r.to_excel(writer, sheet_name='Sheet14')
    PSCOM_r.to_excel(writer, sheet_name='Sheet15')
    u_r.to_excel(writer, sheet_name='Sheet16')
    z_r.to_excel(writer, sheet_name='Sheet17')
    sIM_r.to_excel(writer, sheet_name='Sheet18')
    dIM_r.to_excel(writer, sheet_name='Sheet19')
    sOM_r.to_excel(writer, sheet_name='Sheet20')
    dOM_r.to_excel(writer, sheet_name='Sheet21')

# Model Output

In [8]:
PN_r

GEN                                                      \
t           1            2            3            4            5    
b                                                                    
1  1335.346802  1733.682144  1735.346802  1722.556936  1589.423833   
2     0.000000     0.000000     0.000000     0.000000     0.000000   
3     0.000000     0.000000     0.000000     0.000000     0.000000   

                                                                    \
t           6            7            8            9            10   
b                                                                    
1  1383.795628  1245.513067  1182.506156  1152.166793  1136.919765   
2     0.000000     0.000000     0.000000     0.000000     0.000000   
3     0.000000     0.000000     0.000000     0.000000     0.000000   

                             
t           11           12  
b                            
1  1129.021118  1124.845355  
2     0.000000     0.000000  
3     0.000000     0.000000

In [9]:
POMG_r

Gen                                                              \
t            1           2           3           4           5           6    
n g                                                                           
1 1    0.000000   50.000000  100.000000  150.000000  200.000000  150.000000   
  2    0.000000    0.000000    0.000000  230.708601  391.380310  391.380310   
2 1   50.000000  100.000000  150.000000  200.000000  200.000000  150.000000   
  2  253.707886  253.707886  253.707886  253.707886  350.533803  350.533803   
3 1   50.000000  100.000000  150.000000  200.000000  200.000000  200.000000   
  2  100.189209  100.189209  100.189209  100.189209  146.256002  146.256002   

                                                                             
t            7           8           9           10          11          12  
n g                                                                          
1 1  100.000000  100.000000  150.000000  200.000000  167.985744  200.000000  
  2  391.380310  391.380310  557.199097  557.199097  557.199097  557.199097  
2 1  100.000000   50.000000  100.000000  150.000000  200.000000  200.000000  
  2  350.533803  350.533803  494.483948  494.483948  494.483948  494.483948  
3 1  200.000000  200.000000  200.000000  200.000000  200.000000  200.000000  
  2  146.256002  146.256002  182.849121  182.849121  182.849121  182.849121

In [10]:
PIMG_r

GEN                                                              \
t            1           2           3           4           5           6    
m g                                                                           
1 1  400.000000  400.000000  400.000000  400.000000  400.000000  300.000000   
  2    0.000000    0.000000  160.370959  444.934082  598.601074  590.782364   
2 1  200.000000  200.000000  200.000000  200.000000  200.000000  157.618937   
  2  384.115601  384.115601  384.115601  384.115601  538.954926  538.954926   
3 1  200.000000  200.000000  200.000000  200.000000  200.000000  200.000000   
  2  217.669678  217.669678  217.669678  217.669678  290.241496  290.241496   

                                                                             
t            7           8           9           10          11          12  
m g                                                                          
1 1  200.000000  100.000000  137.464174  200.000000  300.000000  400.000000  
  2  426.697490  607.403564  744.970703  744.970703  744.970703  744.970703  
2 1  107.618937   57.618937  100.000000  150.000000  200.000000  200.000000  
  2  538.954926  538.954926  646.040455  647.106934  647.106934  647.106934  
3 1  200.000000  200.000000  200.000000  200.000000  200.000000  200.000000  
  2  290.241496  290.241496  396.148682  396.148682  396.148682  396.148682

In [11]:
DOM_r

Load                                                      \
t           1            2            3            4            5    
n                                                                    
1     0.000000     0.000000     0.000000  1347.868652  1347.868652   
2  1489.379883  1489.379883  1489.379883  1489.379883  1232.152157   
3     0.000000     0.000000     0.000000     0.000000     0.000000   

                                                                               \
t           6            7            8            9           10          11   
n                                                                               
1  1035.226028   718.774821   646.393423   629.837737  626.050980  625.184841   
2   743.118794   607.902222   570.515158   560.177723  557.319446  556.529140   
3     0.000000  1409.368286  1409.368286  1409.368286  852.331384  670.029936   

               
t          12  
n              
1  624.986730  
2  556.310622  
3  628.896100

In [12]:
DB_r

Load                                                                 \
t           1            2            3            4           5           6    
b                                                                               
1   665.991211   665.991211   665.991211   622.556936  489.423833  447.900758   
2  1269.355591  1269.355591  1269.355591  1032.839949  761.887881  613.721661   
3     0.000000   595.004883   595.004883   595.004883  456.081067  322.173209   

                                                                           
t          7           8           9           10          11          12  
b                                                                          
1  434.950066  430.910856  429.651061  429.258142  429.135594  429.097372  
2  532.699106  488.393093  464.164991  450.916203  443.671295  439.709522  
3  277.863895  263.202207  258.350741  256.745420  256.214229  256.038461

In [13]:
PL_r

Flow                                                              \
t          1           2           3           4           5           6    
l                                                                           
1  669.355591  669.355591  669.355591  700.000000  700.000000  613.721661   
2    0.000000  398.335342  400.000000  400.000000  400.000000  322.173209   
3  600.000000  600.000000  600.000000  332.839949   61.887881    0.000000   
4    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
5    0.000000  196.669540  195.004883  195.004883   56.081067    0.000000   
6    4.589844   18.842163  500.000000  600.000000  600.000000  493.845718   
7  600.000000  600.000000  564.306342  600.000000  600.000000  499.637217   
8    0.000000  231.801300    0.000000  229.217991  209.825065    0.000000   

                                                                           
t          7           8           9           10          11          12  
l                                                                          
1  532.699106  488.393093  464.164991  450.916203  443.671295  439.709522  
2  277.863895  263.202207  258.350741  256.745420  256.214229  256.038461  
3    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000  
4    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000  
5    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000  
6  227.394511  314.662476  502.088198  562.294312  600.000000  259.868774  
7  447.270884  351.520005  565.693775  512.835499  462.045192    0.000000  
8  600.000000  600.000000  600.000000  409.141388  410.376892  246.046978

In [14]:
deltaIM_r

Angle                                                              \
t          1           2           3           4           5           6    
m                                                                           
1 -146.828788 -134.960644  265.707539   96.739436 -128.886885 -184.998753   
2  499.630041  499.630041  469.907334  499.630041  499.630041  416.056271   
3    0.000000  108.366166  -86.737938  104.135719   -8.416583 -178.852682   

                                                                           
t          7           8           9           10          11          12  
m                                                                          
1 -328.163489 -212.450576  -32.840370   30.165503   68.602110 -210.781980  
2  372.449950  292.716590  471.062673  427.046702  384.752763    0.000000  
3  345.375430  353.514786  356.208050  198.168075  199.491788   62.749096

In [15]:
deltaOM_r

Angle                                                              \
t          1           2           3           4           5           6    
n                                                                           
1 -150.650828 -150.650828 -150.650828 -402.890605 -628.516925 -596.232347   
2    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
3    0.000000  -84.658656  -86.737938  -86.737938 -183.141427 -178.852682   

                                                                           
t          7           8           9           10          11          12  
n                                                                          
1 -517.518703 -474.475285 -450.937614 -438.066379 -431.027931 -427.179057  
2    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000  
3 -154.254610 -146.115255 -143.421990 -142.530805 -142.235917 -142.138341

In [16]:
deltaB_r

Angle                                                              \
t          1           2           3           4           5           6    
b                                                                           
1    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
2 -650.280869 -650.280869 -650.280869 -680.052000 -680.052000 -596.232347   
3    0.000000 -221.133670 -222.057796 -222.057796 -222.057796 -178.852682   

                                                                           
t          7           8           9           10          11          12  
b                                                                          
1    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000  
2 -517.518703 -474.475285 -450.937614 -438.066379 -431.027931 -427.179057  
3 -154.254610 -146.115255 -143.421990 -142.530805 -142.235917 -142.138341

In [17]:
EIM_r

Storage Level                                                              \
t            1           2           3           4           5           6    
m                                                                             
1    507.547485  507.547485  507.547485  151.494005    0.000000    0.000000   
2    917.950955  683.179226  464.243408  285.343594   66.273499    0.000000   
3    145.147731  344.148586  344.148586  552.998409  555.241813  628.560875   

                                                                             
t          7           8           9             10          11          12  
m                                                                            
1    0.000000    0.000000    0.000000  0.000000e+00    0.000000   57.547485  
2    0.000000    0.000000   47.081492  4.708149e+01  138.517031  322.146451  
3  891.845986  548.250721  210.123698  3.441691e-15    0.000000    0.000000

In [18]:
EOM_r

Storage Level                                                     \
t            1            2            3            4           5    
n                                                                    
1   2272.160241  1610.593401  1020.418026  1020.418026  242.640249   
2   2400.000000  1749.253337  1154.062229   574.767056   90.687059   
3    110.876469   246.046757   457.835629   507.501523  808.463609   

                                                                   \
t           6            7            8           9            10   
n                                                                   
1     0.000000     0.000000     0.000000  143.684427   665.189029   
2     0.000000   231.347003   492.259222  655.644007  1195.644007   
3  1258.463609  1570.094010  1055.524805  540.955600    67.045417   

                             
t           11           12  
n                            
1  1289.287214  1919.287214  
2  1735.644007  2275.644007  
3     0.000000   110.876469

In [19]:
z_r

Connection                                                       
t         1    2    3    4    5    6    7    8    9    10   11   12
n                                                                  
1       -0.0 -0.0 -0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
2        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
3        0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0

In [20]:
u_r

Connection                                                       
t         1    2    3    4    5    6    7    8    9    10   11   12
b                                                                  
1        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
2        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
3        0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0